encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [25]:
import numpy as np
import os
import sys
# 获取当前文件所在的文件夹路径
if "__file__" in globals():
    # 获取__file__变量的值
    file_path = __file__
    # 获取当前文件所在的文件夹路径
    dir_now = os.path.dirname(file_path)
else:
    # 获取当前工作目录
    dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [26]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_PLUTO
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
# for method in [METHOD_MITHRAL, METHOD_PQ]:

In [27]:
linear_name = 'etl1'
feedback_bits = 256
linear_name_full = "ex_linear1"

auto_train_change_nbits = False # 是否根据已运行的训练性能结果改变nbits自动训练，（train_sam_num取已训练的最大值）
auto_train_change_upcast = False # 是否根据已运行的训练性能结果改变upcast自动训练，（train_sam_num取已训练的最大值）

if auto_train_change_nbits:
    nbits_trained = 8
if auto_train_change_upcast:
    upcast_trained = 16
quantize_lut = False
nbits_goal = 8
upcast_goal = -1
lut_work_const = -1
if quantize_lut == False:
    nbits_goal = 0
nbits = nbits_goal # 要运行的量化比特数
upcast_every = upcast_goal # 要运行的upcast

test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)

if not auto_train_change_nbits and not auto_train_change_upcast:
    ncodebooks = 16 # max:64
    ncentroids = 32
    train_sam_num = 1000 # 训练集样本数
elif auto_train_change_nbits:
    param2change = "nbits"
    param_trained = nbits_trained
    param_goal = nbits_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "upcast_every", 16)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])
elif auto_train_change_upcast:
    param2change = "upcast_every"
    param_trained = upcast_trained
    param_goal = upcast_goal
    cb_ct_ntr_combinations_unique = change_param_auto_run_list(linear_name, method, feedback_bits, param2change, param_trained, param_goal, "nbits", 8)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])

batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [28]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
create_dir(AMM_train_dirs["dir_result"])


In [29]:
dataset_prepare(AMM_train_dirs["dir_joined"], linear_name_full, feedback_bits, [train_sam_num, test_sam_num], 
                batch_size, S1 = S1_dict[linear_name])

In [30]:
if method == METHOD_PLUTO:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut, 
                        upcast_every=upcast_every, bias_path=AMM_train_dirs["biaspath"],lut_work_const=-1)
elif method == METHOD_MITHRAL:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every, lut_work_const=lut_work_const)
else:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut,
                        upcast_every=upcast_every)

running method:  Mithral
X.shape:  (1024000, 64)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    26366.85965300611
learn_multisplits(): returning loss:  6337.534834518528
learn_multisplits(): initial loss:    40354.50194061942
learn_multisplits(): returning loss:  11106.161188978904
learn_multisplits(): initial loss:    142946.7497124098
learn_multisplits(): returning loss:  40121.73077704517
learn_multisplits(): initial loss:    158806.95212636088
learn_multisplits(): returning loss:  36488.16175538957
learn_multisplits(): initial loss:    47767.72875959566
learn_multisplits(): returning loss:  11823.196660843763
learn_multisplits(): initial loss:    19503.756272698
learn_multisplits(): returning loss:  5994.446205890536
learn_multisplits(): initial loss:    12994.871381561192
learn_multisplits(): returning loss:  3944.5388247045657
learn_multisplits(): initial loss:    15487.575316472636
learn_multisplits(): returning loss:  4144.622815484079
learn_mu

/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../matmul.py:330: UserWarning: Persisting input arguments took 2.41s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  est = _fitted_est_for_hparams(


In [31]:
x_test = np.load(AMM_train_dirs["dir_test"]+'/'+AMM_train_dirs["linearin_path_test"])
w_test = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["weightpath"])
bias = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["biaspath"])
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
if method == METHOD_PLUTO:
    y_out_last = y_out_matmul
else:
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [32]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), 
                                                        y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL or method == METHOD_PLUTO:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i_lwc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every, lut_work_const)), y_out_last_re)
elif method == METHOD_PQ or method == METHOD_MITHRALPQ:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i_ql%i_nb%i_uc%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids, quantize_lut, nbits, upcast_every)), y_out_last_re)
else:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids)), y_out_last_re)

[[-0.40914336 -0.38542745 -0.46585375 ... -0.2561478   1.2336944
  -0.3459559 ]
 [-0.40910152 -0.38369647 -0.4018714  ... -0.24560563  1.1985148
  -0.3349195 ]
 [-0.36242256 -0.37482992 -0.43365267 ... -0.251956    1.2427292
  -0.34156224]
 ...
 [-0.18052287 -0.1548559  -0.43180403 ... -0.355245    0.707908
  -0.30527756]
 [-0.29199627 -0.2874528  -0.3661354  ... -0.21753731  0.7594963
  -0.26501322]
 [-0.22232394 -0.25397086 -0.21779217 ... -0.00214468  0.6389945
  -0.17712714]]
y_out_last.shape:  (1024000, 512)
y_out_last_re.shape:  (1000, 32, 32, 512)
